In [6]:
import mt.ml_funcs
import time
import pandas as pd
from pathlib import Path
import mt.resources.indicators as ind
import mt.resources.binance_funcs as funcs
import mt.resources.features as features
import numpy as np
import json
from itertools import product
from collections import Counter
from datetime import datetime
from pyarrow import ArrowInvalid
if not Path('/pi_2.txt').exists():
    from sklearnex import patch_sklearn
    patch_sklearn()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import confusion_matrix, roc_auc_score, make_scorer
from sklearn.inspection import permutation_importance
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
